In [ ]:
import time
import datetime 
import numpy as np
from random import gauss, seed
from math import sqrt, exp 
seed(12345678)

https://en.wikipedia.org/wiki/It%C3%B4%27s_lemma

$$ dS/S: \textit{asset return}$$
$$ dt: \textit{time unit (assume year)}$$
$$ \mu: \textit{expected return per year} $$
$$ \sigma: \textit{return volatility per year} $$
$$ S: \textit{asset price per unit} $$
$$ dz \sim N(0,dt) \sim \epsilon \sqrt{dt}$$
$$ \epsilon \sim N(0,1)$$
$$\textit{Assume:   } dS_t/S_t = \mu_t dt + \sigma_t dz_t $$
By Ito's lemma:

$$ d\log{S_t} = (\mu_t - \sigma_t^2/2) dt + \sigma_t dz_t$$
$$ \int_0^T d\log{S_t} = \int_0^T (\mu - \sigma^2/2) dt + \sigma dz_t$$

$$ \log S_T - \log S_0 = (\mu_t - \sigma_t^2/2)T + \sigma \epsilon \sqrt{T}$$


$$ S_T = S_0 \exp((\mu_t - \sigma_t^2/2) T + \sigma_t \epsilon \sqrt{T}) $$

If we assume risk-neutral measure, 

$$\mu_t = r_t - q_t$$

$r$ is risk-free rate, and $q$ is dividend yield per year

In [ ]:
def generate_asset_price(S_0,v,mu,T,z): # simulate asset prices 
  S_T = S_0*exp((mu-v**2/2)*T + v*z*sqrt(T))
  return S_T

def call_payoff(S_T,K): # European call option for the time being
  return max(0., S_T-K)

In [ ]:
S = 100.    # underlying asset price (S_0) 
v = 0.20978 # volatility per year
r = 0.01    # risk free rate per year
q = 0.001   # dividend yield per year
mu = r - q  # expected under risk neutral measure 
T = (datetime.date(2021,11,30) - datetime.date(2020,10,30)).days/365.
K = 110.    # exercise price

In [ ]:
z = np.random.standard_normal((1,1))
S_T = generate_asset_price(S,v,mu,T,z)
print(S_T)
print(call_payoff(S_T,K))

69.60737409800016
0.0


Our goal is to simulate a lot of call option payoff, and then average it out, and then discount the average back to the present. This will give us the derivative price.

In [ ]:
simulations = 10**4
payoffs = []
t = time.time()
for i in range(simulations):
  z = np.random.standard_normal((1,1))
  S_T = generate_asset_price(S,v,mu,T,z)
  DS_T = call_payoff(S_T,K)
  payoffs.append(DS_T)
  # antithetic variate method

elapsed = time.time()-t
discount_factor = exp(-r*T)
price = discount_factor*np.average(payoffs)
print("Derivative price (average):", round(price,4))
print("Time and std:", round(elapsed,4), np.std(payoffs))

Derivative price (average): 5.3196
Time and std: 0.0677 11.89475219969843


Class quiz: Use antithetic variate method to compute the derivative price. Report your code and results. Write down your name and student id in the solution. 

Longstaff, F. A., & Schwartz, E. S. (2001). Valuing American options by simulation: a simple least-squares approach. The review of financial studies, 14(1), 113-147.

In [ ]:
# we need to learn polyfit function first 
# https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html

x = [0., 1., -1.]
y = [10., 12., +12.]
z = np.polyfit(x,y,2)
print(z)
p = np.poly1d(z)
p(30)

[ 2.  0. 10.]


1809.999999999998

In [ ]:
temp = np.random.standard_normal((4,3))
print(temp)
print("\n")
print(np.cumsum(temp, axis=0))
print("\n")
print(np.cumsum(temp, axis=1))

[[-1.10128104  0.92053321 -2.03663438]
 [-1.0211612  -0.07456679 -0.0429232 ]
 [ 0.15611026 -0.81319573  0.55357906]
 [-0.79242163  1.87324108 -1.01045775]]


[[-1.10128104  0.92053321 -2.03663438]
 [-2.12244224  0.84596642 -2.07955759]
 [-1.96633197  0.03277069 -1.52597853]
 [-2.7587536   1.90601177 -2.53643628]]


[[-1.10128104 -0.18074783 -2.21738221]
 [-1.0211612  -1.09572799 -1.13865119]
 [ 0.15611026 -0.65708547 -0.10350641]
 [-0.79242163  1.08081945  0.0703617 ]]


In [ ]:
# Longstaff & Schwartz algorithm
import math
import numpy as np
from math import exp, log, sqrt 

np.random.seed(123456)

# Inputs 
S0 = 100.       # underlying asset price (S_0) 
sigma = 0.20978 # volatility per year
r = 0.01        # risk free rate per year
div = 0.001     # dividend yield per year
T = 1.          # time to maturity
K = 110.        # exercise price

# simulation parameters input
I = 10**3       # # of simulation (The larger, the more accurate)
M = 12          # time steps (The larger, the more accurate)

In [ ]:
dt = T/M        # length of time step 
df = math.exp(-r*dt) # discount factor 
mu = r - div    # expected under risk neutral measure 
return_series = (mu - sigma**2/2.)*dt + \
                 sigma*sqrt(dt)*np.random.standard_normal((M,I))
print(return_series.shape)
S = S0*np.exp(np.cumsum(return_series,axis=0))
# print(S)

(12, 1000)


In [ ]:
# compute inner values
h = np.maximum(S - K,0.) # put (K-S) or call (S-K) should be determined here
V = h[-1]  # terminal value 

# Let us compute the value of American option with backward induction
for t in range(M-1,0,-1):
  rg = np.polyfit(S[t],V*df,10) # estimate contiuation-value function with stock prices
  C = np.polyval(rg,S[t])   # evaluate continuation value with stock prices with the function
  V = np.where(h[t]>C,h[t],V*df)

V0 = df*np.average(V)
print("American call option value is", round(V0,4))



American call option value is 5.1978
